# Errors - Monte Carlo simulations 

## In this project, I will show how to extract errors on best-fit parameters estimated from the curve fitting techniques introduced in  __[curve_fitting](https://github.com/fakahil/Projects/blob/master/curve_fitting.ipynb)__. I will use the same example (the limb data) for illustration.

In [1]:
## importing the libraries
import numpy as np
import math as m
import scipy
import matplotlib.pyplot as plt
from scipy import special
from scipy.optimize import curve_fit

In [2]:
def Erfc(x,sigma):

 y = special.erfc(x/(sigma*np.sqrt(2)))
 return y

## Introducing the model to be used later for the fitting

def SL_fit(x,w1,w2,w3,s1,s2,s3):

  f = 0.5*(w1*Erfc(x,s1)+w2*Erfc(x,s2)+w3*Erfc(x,s3)+ (1-w1-w2-w3))
  return f

### From now on I will use __[scipy.optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)__  for the fitting problem

In [3]:
## Jacobian matrix
def DF(x,w1,w2,w3,s1,s2,s3):
 dfw1 = 0.5*(Erfc(x,s1)-1)
 dfw2 = 0.5*(Erfc(x,s2)-1)
 dfw3 =  0.5*(Erfc(x,s3)-1)
 dfs1 = w1*x*np.exp(-x**2/(2*s1**2))/ (np.sqrt(2*np.pi)*s1**2)
 dfs2 = w2*x*np.exp(-x**2/(2*s2**2))/ (np.sqrt(2*np.pi)*s2**2)
 dfs3 = w3*x*np.exp(-x**2/(2*s3**2))/ (np.sqrt(2*np.pi)*s3**2)
 return np.transpose(np.array([dfw1,dfw2,dfw3,dfs1,dfs2,dfs3]))


In [4]:
## Loading the data:
path = "/home/fatima/Desktop/project_3/"
file = np.loadtxt(path+'limb_profile_av_norm_shifted')
x = file[:,0]
y = file[:,1]

In [5]:
ind = np.where(x>=0)
x = x[ind]
y = y[ind]
weights = np.sqrt(np.abs(y)) ## Poisson weighting

In [6]:
p0=[0.3, 0.3, 0.2, 1, 2, 3] ## initial guess best-fit parameters
popt, pcov = curve_fit(SL_fit,x,y,p0,method='lm',sigma=weights,jac=DF,ftol=1e-8,xtol=1e-8,maxfev=5000)
chi_sq_w = np.sum((1/weights**2)*(SL_fit(x,*popt)-y)**2)
red_chi_sq = chi_sq_w/(len(y)-len(popt))
print popt # to print the best-fit parameters

[ 0.52750103  0.28882569  0.10191755  0.25905336  0.76540583  2.83343005]


### In the lines above, popt is the array of best-fit parameters, while pcov is the covariance matrix, the diagonal of which can give an estimate of the errors by typing the following:

In [7]:
perr = np.sqrt(np.diag(pcov))
print perr ## to print the error

[ 0.00349415  0.0028457   0.00119396  0.00135734  0.00715294  0.04738134]


### One can also use another method to compute the errors on the best-fit parameters. The so called "Monte Carlo" method. 

In [8]:
### The "true" solution is assumed to be that returned by curve_fit, popt
w10, w20, w30, s10, s20, s30 = popt
w40 = 1-w10-w20-w30

In [9]:
### we will make a copy for our data set so we don't alter the original data set
xr = x.copy()
yr = y.copy()
ery = weights.copy()

In [10]:
N = len(y)
trials = 1000 ## number of synthetic data sets
omegas1, omegas2, omegas3,omegas4,sigmas1,sigmas2, sigmas3 = [],[],[],[],[],[],[] #defining the arrays of best-fit parameters corresponding to the number of trials "trials"
chi_s = [] ## the array of dimension "trials" of chi-square values 

### Now generating synthetic data sets using  __[numpy.random.randint](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.randint.html)__. 

In [11]:
from numpy.random import normal, randint

In [12]:
for i in range(trials):       
 indx = randint(0, N, N)    
 xr[:] = x[indx]
 yr[:] = y[indx]
 ery[:] = weights[indx]  
 ok = (xr != xr[0]).any()

 if (not ok):
   print "All elements are the same. Invalid sample."
   print xr, yr
 else:
    
    ## now preforming the fit using curve_fit on the synthetic data sets and saving results to popti
   popti, pcovi = curve_fit(SL_fit,xr,yr,p0,method='lm',sigma=ery,ftol=1e-8,xtol=1e-8,maxfev=5000)
   chi_sq_wi = np.sum((1/ery**2)*(SL_fit(xr,*popti)-yr)**2) ## chi_squared values
   red_chi_sqi = chi_sq_wi/(len(yr)-len(popti)) ## reduced chi_squared values
   om1, om2,om3,sig1,sig2,sig3 = popti
   om4 = 1-om1-om2-om3
## saving best-fit parameters and reduced chi_sq values
   omegas1.append(om1)
   omegas2.append(om2)
   omegas3.append(om3)
   omegas4.append(om4)
   sigmas1.append(sig1)
   sigmas2.append(sig2)
   sigmas3.append(sig3)
   chi_s.append(red_chi_sqi)
   print popti ##to print the output for each trial 

[ 0.51780298  0.29471219  0.10472405  0.25423208  0.74833391  2.72665156]
[ 0.52129061  0.29353422  0.10319298  0.2550758   0.75857336  2.79692301]
[ 0.51306274  0.30034848  0.10439981  0.25379152  0.73951306  2.77894058]
[ 0.53365379  0.28457255  0.10031272  0.26452477  0.7753637   2.87604064]
[ 0.52833911  0.28897794  0.10118776  0.2593886   0.77123206  2.86117997]
[ 0.52448991  0.29194521  0.10179544  0.25803698  0.76412923  2.84068826]
[ 0.54145759  0.27780062  0.09965041  0.26467896  0.78719177  2.91283197]
[ 0.53095848  0.28561657  0.10171466  0.26005834  0.76725018  2.8396865 ]
[ 0.52902465  0.28694202  0.10166209  0.25976067  0.76918532  2.79408516]
[ 0.54044506  0.27784204  0.1002904   0.26247875  0.78540453  2.8818541 ]
[ 0.52980677  0.2848685   0.10290806  0.25969338  0.76599433  2.77238101]
[ 0.52936404  0.28811663  0.10103488  0.25763627  0.77464682  2.86450959]
[ 0.52031571  0.29408812  0.10352346  0.25716309  0.75133869  2.79816134]
[ 0.53425524  0.28351565  0.10073671  

/usr/local/lib/python2.7/dist-packages/scipy/optimize/minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


[ 0.28731485  0.5292576   0.10167048  0.76798473  0.26162477  2.83761925]
[ 0.5247599   0.29226996  0.10126959  0.2578713   0.76648919  2.84495703]
[ 0.52591027  0.28932518  0.10281447  0.26021156  0.75974466  2.81205229]
[ 0.52037307  0.29353716  0.10386839  0.25815326  0.74868591  2.77357637]
[ 0.52476081  0.29207564  0.10170637  0.25701154  0.76487136  2.86056447]
[ 0.53456731  0.28252339  0.10129951  0.26120784  0.77489021  2.84490542]
[ 0.09315831  0.61875699  0.19572435  0.00336271  0.37164124  1.67974338]
[ 0.53191365  0.28413132  0.1019134   0.26093346  0.77191662  2.81049658]
[ 0.53334859  0.2847749   0.10058468  0.2608581   0.7765642   2.8836232 ]
[ 0.5260746   0.28839298  0.1031883   0.25905232  0.76100372  2.77683836]
[ 0.52388349  0.29319221  0.10129585  0.25735639  0.76616122  2.84936941]
[ 0.53808165  0.28137813  0.09941461  0.26333898  0.78439046  2.90885228]
[ 0.52392466  0.29098829  0.10279606  0.25801993  0.75619837  2.79025982]
[ 0.52959063  0.28844162  0.10051073  

[ 0.52630209  0.29025312  0.10192653  0.25712975  0.76378061  2.85159543]
[ 0.52711995  0.28821072  0.10258416  0.25780268  0.76155858  2.80302378]
[ 0.52336361  0.29202943  0.10245     0.2575119   0.76241662  2.79057595]
[ 0.54146252  0.27809212  0.0993454   0.26588019  0.79170102  2.91019014]
[ 0.52038995  0.29531919  0.10221233  0.25669747  0.75747156  2.80588171]
[ 0.52541899  0.28973083  0.10301623  0.25844946  0.75809407  2.81187752]
[ 0.5319576   0.28547726  0.10119652  0.25996102  0.77422179  2.87288352]
[ 0.52876107  0.2878591   0.10140257  0.25952417  0.77100842  2.82419938]
[ 0.07127846  0.6461383   0.19080611  0.00426523  0.36587666  1.71109834]
[ 0.5152671   0.2993416   0.10315691  0.25510515  0.7460802   2.7909378 ]
[ 0.52766433  0.28694785  0.10321582  0.26016951  0.7619919   2.78499356]
[ 0.52873652  0.28807465  0.1017274   0.25825151  0.77044067  2.85694999]
[ 0.52772129  0.28829933  0.10198788  0.25747681  0.76664463  2.81820052]
[ 0.5374271   0.2831179   0.09885107  

[ 0.52710912  0.28866884  0.10225792  0.25977138  0.76251926  2.81431172]
[ 0.29357765  0.5198607   0.10431444  0.74849487  0.25471892  2.7679751 ]
[ 0.52812251  0.28631115  0.10337887  0.25882837  0.75915733  2.77952096]
[ 0.53380455  0.28363436  0.10126716  0.26353256  0.77217167  2.87965068]
[ 0.51787947  0.2959151   0.10364456  0.25511938  0.75030034  2.75580938]
[ 0.53542145  0.28052517  0.10220858  0.26070359  0.77393372  2.81172927]
[ 0.52157414  0.29259138  0.10379363  0.25732786  0.75233293  2.79156025]
[ 0.52524862  0.28952025  0.10277273  0.2589276   0.75897665  2.77010331]
[ 0.51879567  0.29688778  0.10253273  0.25609551  0.75119801  2.82765493]
[ 0.52838924  0.28842424  0.1015291   0.25965788  0.76638238  2.84425995]
[ 0.51848156  0.29426153  0.10448664  0.2570702   0.74655656  2.7317089 ]
[ 0.53275545  0.2850493   0.10071207  0.25998526  0.77509839  2.87016247]
[ 0.52696821  0.2882145   0.10294566  0.25955236  0.75941763  2.81822776]
[ 0.52323598  0.29145984  0.10304685  

[ 0.52150464  0.29329757  0.10322982  0.25630324  0.7541301   2.8049907 ]
[ 0.52943189  0.28759912  0.10131672  0.26058899  0.76979325  2.84341573]
[ 0.52467066  0.28902021  0.10393133  0.25936036  0.75562758  2.75994296]
[ 0.29016975  0.52641255  0.10172961  0.76564431  0.25814047  2.83779598]
[ 0.52764056  0.28712111  0.10288766  0.25837719  0.76325159  2.78326046]
[ 0.52251538  0.29322779  0.10272077  0.25849025  0.75459738  2.84152354]
[ 0.51934069  0.29635616  0.10227989  0.25485911  0.75399251  2.80838859]
[ 0.5275772   0.28999446  0.10085656  0.25726157  0.7683976   2.85868449]
[ 0.52986175  0.28694753  0.10169817  0.25823994  0.77065629  2.85383855]
[ 0.53202021  0.28579661  0.10091513  0.26114791  0.77368294  2.87810319]
[ 0.52495193  0.28910547  0.10363415  0.25766505  0.75673946  2.77229637]
[ 0.52437399  0.29083191  0.1027135   0.25749929  0.7611234   2.80254148]
[ 0.5288668   0.28732595  0.10166389  0.25953327  0.76528946  2.81020385]
[ 0.53241343  0.2861656   0.10041337  

[ 0.5276119   0.28782597  0.10259367  0.2572721   0.76539106  2.80453567]
[ 0.5184783   0.2957683   0.10342041  0.25637218  0.7482443   2.77971043]
[ 0.53659265  0.28030156  0.10145713  0.26350625  0.77644914  2.84416036]
[ 0.52694306  0.29077124  0.10095033  0.25709491  0.77148102  2.88062678]
[ 0.28553821  0.53179726  0.10127819  0.77293691  0.25916142  2.86268261]
[ 0.52372652  0.29231586  0.10192659  0.25675985  0.75929218  2.81675714]
[ 0.52945158  0.28572226  0.10299203  0.2603878   0.76284122  2.8131133 ]
[ 0.5253707   0.28999668  0.10260932  0.25689112  0.76092178  2.81321853]
[ 0.5264749   0.29116517  0.10099802  0.25768648  0.77019861  2.87666189]
[ 0.52310969  0.29066315  0.10381287  0.2582183   0.75501916  2.76364245]
[ 0.53266301  0.28664493  0.09972338  0.26140692  0.77815279  2.91808681]
[ 0.51056008  0.29889078  0.10714799  0.25334157  0.73187801  2.65665704]
[ 0.52433398  0.29049999  0.10295249  0.2572683   0.76025743  2.78871038]
[ 0.52280189  0.29384601  0.10179031  

[ 0.52351879  0.29626609  0.09950466  0.2564797   0.76993322  2.94174237]
[ 0.52650822  0.29025492  0.10137817  0.25762226  0.76369918  2.83892077]
[ 0.52337322  0.29063413  0.10344697  0.25726244  0.75779065  2.75972312]
[ 0.52598605  0.29006452  0.10211042  0.25907657  0.76343466  2.82290879]
[ 0.52813618  0.28860162  0.10189258  0.26104123  0.76746272  2.85927847]
[ 0.28049229  0.53853096  0.09973483  0.78503468  0.2622274   2.89719719]
[ 0.52544327  0.29076398  0.10204194  0.25836822  0.76310707  2.8313018 ]
[ 0.52026506  0.29354963  0.10395558  0.25723144  0.74965528  2.77420767]
[ 0.5247171   0.29069113  0.102528    0.25828391  0.75921436  2.81286479]
[ 0.52684543  0.28971534  0.10164376  0.25721073  0.76559197  2.83531795]
[ 0.52594202  0.29017062  0.10228079  0.25635112  0.76428463  2.84107937]
[ 0.5289056   0.28949642  0.1002741   0.25858292  0.77251406  2.88825832]
[ 0.51852037  0.29562303  0.1037344   0.25563593  0.75178676  2.78908453]
[ 0.52902934  0.28678077  0.10213886  

[ 0.52929645  0.28793194  0.10125864  0.25869101  0.76911369  2.86126487]
[ 0.53320016  0.28489256  0.10044565  0.26159278  0.78034917  2.87307599]
[ 0.52574764  0.29001368  0.10225555  0.25921034  0.75901415  2.81562831]
[ 0.09923129  0.63191703  0.17771703  0.006538    0.39395246  1.78767132]
[ 0.53595576  0.28208098  0.1005969   0.26373088  0.77891398  2.87849982]
[ 0.51861492  0.29290955  0.10551099  0.2567067   0.74147035  2.70432719]
[ 0.53107854  0.2876433   0.10005808  0.25938022  0.77827878  2.89729478]
[ 0.52160072  0.29316155  0.10335265  0.25581587  0.75296115  2.80704465]
[ 0.52931247  0.28933322  0.0999779   0.25871824  0.7811064   2.88256764]
[ 0.53189274  0.28527411  0.10125389  0.26048375  0.77451064  2.85818027]
[ 0.52505319  0.28827716  0.10401935  0.25898918  0.75466654  2.74698268]
[ 0.5272063   0.28732839  0.10334876  0.25948526  0.75886962  2.78702641]
[ 0.55246005  0.26936611  0.09770425  0.27064095  0.80881942  2.97680152]
[ 0.52519719  0.28989871  0.10297964  

[ 0.53096013  0.28599651  0.10132206  0.26102624  0.7733681   2.83904765]
[ 0.53295435  0.28550362  0.1001244   0.25935821  0.78119616  2.87742617]
[ 0.52911002  0.28912229  0.10025344  0.25713488  0.77551707  2.87747989]
[ 0.52565747  0.29070289  0.10185273  0.25756771  0.76286919  2.83515279]
[ 0.53157627  0.2870114   0.10018012  0.25702632  0.77541539  2.89112248]
[ 0.52771203  0.28719687  0.10299977  0.25861102  0.76124833  2.79532915]
[  7.13864084e-02   6.43552443e-01   1.92243668e-01   9.90038755e-04
   3.62271221e-01   1.66563598e+00]
[ 0.52841079  0.28881544  0.10116992  0.26066672  0.76957296  2.85255336]
[ 0.53748314  0.28281583  0.09890411  0.26384694  0.78806422  2.94016195]
[ 0.53089353  0.28424391  0.10266871  0.26077172  0.76497477  2.79377089]
[ 0.52468294  0.29323155  0.10080495  0.25844071  0.7644761   2.88374157]
[ 0.52856263  0.28666958  0.10266642  0.26124391  0.76189338  2.80060798]
[ 0.52187814  0.2926676   0.10338985  0.25749876  0.75390421  2.79575108]
[ 0.532

  ### Now let's create arrays of the fluctuations of the best fit parameters popti around the true solution popt0

In [13]:
omegas1 = np.array(omegas1) - w10
omegas2 = np.array(omegas2) - w20
omegas3 = np.array(omegas3) - w30
omegas4 = np.array(omegas4) - w40
sigmas1 = np.array(sigmas1) - s10
sigmas2 = np.array(sigmas2) - s20
sigmas3 = np.array(sigmas3) - s30

In [14]:
## now computing the standard deviation of the best-fit parameters population
stdw1, stdw2,stdw3,stdw4,stds1,stds2,stds3 = omegas1.std(), omegas2.std(), omegas3.std(),omegas4.std(), sigmas1.std(),sigmas2.std(),sigmas3.std()
print "Bootstrap errors in w1,w2,w3,w4,s1,s2,s3 are:", stdw1, stdw2,stdw3,stdw4,stds1,stds2,stds3

Bootstrap errors in w1,w2,w3,w4,s1,s2,s3 are: 0.0772531005887 0.0610754089333 0.0149780096104 0.00644211083616 0.121609755555 0.0840172033095 0.192045426395
